In [ ]:
%conda install langchain langchainhub transformers chromadb sentence-transformers

In [ ]:
# !sudo apt update && sudo apt install -y \
#   libmagic-dev \
#   poppler-utils \
#   tesseract-ocr \
#   qpdf \
#   libreoffice \
#   pandoc

In [ ]:
!pip install "langchain-unstructured[local]" langchain_community langchain_huggingface 
# !pip install unstructured

In [2]:
# import torch
import subprocess
import getpass
# from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain.text_splitter import RecursiveCharacterTextSplitter
# # from sentence_transformers import SentenceTransformer
# from langchain_community.vectorstores.utils import filter_complex_metadata
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# # import numpy as np


In [28]:
#danh sach cac model

#TinyLlama/TinyLlama_v1.1
#TinyLlama/TinyLlama-1.1B-Chat-v1.0
#meta-llama/Llama-3.2-1B-Instruct
#meta-llama/Llama-3.2-1B
huggingface_model_name = "meta-llama/Llama-3.2-1B-Instruct"
embedding_model_name = "all-MPNet-base-v2"

In [29]:
huggingface_api_key = getpass.getpass()
subprocess.run(["huggingface-cli", "login", "--token", huggingface_api_key]);

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `learning` has been saved to /home/bach/.cache/huggingface/stored_tokens
Your token has been saved to /home/bach/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [9]:
# import chromadb
# from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain.text_splitter import RecursiveCharacterTextSplitter

# === STEP 1: Load văn bản cần indexing ===
def load_documents():
    return [
        "Hồ Chí Minh là một trong những lãnh tụ vĩ đại nhất của dân tộc Việt Nam.",
        "Python là một ngôn ngữ lập trình rất phổ biến và dễ học.",
        "Mặt Trăng là vệ tinh tự nhiên duy nhất của Trái Đất.",
        "Việt Nam nổi tiếng với các món ăn như phở, bún chả và bánh mì."
    ]


# === STEP 2: Chia nhỏ bằng RecursiveTextSplitter ===
def chunk_documents(docs):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=20
    )
    chunks = []
    for doc in docs:
        chunks.extend(splitter.split_text(doc))
    return chunks


# === STEP 3: Index với ChromaDB ===
def build_vector_db(chunks):
    embedding_func = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
    client = chromadb.Client()
    collection = client.create_collection(name="rag-demo1", embedding_function=embedding_func)

    for i, chunk in enumerate(chunks):
        collection.add(
            documents=[chunk],
            ids=[str(i)],
            metadatas=[{"source": f"chunk_{i}"}]
        )

    return collection


# === STEP 4: Truy vấn bằng Chroma ===
def retrieve_similar_chunks(query, collection, top_k=3):
    results = collection.query(query_texts=[query], n_results=top_k)
    return results["documents"][0]  # danh sách văn bản liên quan


# === STEP 5: Tạo prompt RAG và sinh kết quả từ LLaMA ===
def generate_answer(query, retrieved_chunks, llm_pipeline):
    context = "\n".join(retrieved_chunks)
    prompt = f"""Bạn là một trợ lý AI. Hãy trả lời câu hỏi dựa trên thông tin sau:
{context}

Câu hỏi: {query}
Trả lời:"""

    result = llm_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)
    return result[0]["generated_text"]

In [12]:
docs = load_documents()
chunks = chunk_documents(docs)

print("[+] Tổng số chunk:", len(chunks))

#     # Embedding model (multi-language, nhẹ và tốt)
# embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# index, chunks, _ = build_vector_db(chunks, embed_model)

[+] Tổng số chunk: 4


In [ ]:
    # Query user
query = input("Nhập câu hỏi của bạn: ")
similar_chunks = retrieve_similar_chunks(query, index, chunks, embed_model)
result = generate_answer(query, similar_chunks, llm)
print("\n[+] Câu trả lời:\n", result)
